In [1]:
# libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import numpy as np
import re

warnings.simplefilter(action='ignore', category=FutureWarning) # ignores warnings from pd.append()

In [7]:
# column names
col_names = ['Titel', 'Bevilliget beløb', 'Modtager', 'Institution', 'Virkemidler', 'Område', 'År', 'Beskrivelse']

# create an Empty DataFrame object
df = pd.DataFrame(columns = col_names)

In [8]:
URL = "https://dff.dk/forskningsprojekter?b_start:int="

for i in range(0, 4440, 10):

    print("Scraping page: "+ str(int(float(i/10)))) # prints page being scraped at the moment

    page = requests.get(URL+str(i))
    soup = BeautifulSoup(page.text, "html.parser")

    # Load page as html
    bevillinger = soup.find_all("div", class_ = "result-item")

    # Retrieve all grants from first page
    for bevilling in bevillinger:
        titel = bevilling.find("h2", class_ = "result-title").text.strip()
        beløb = bevilling.find("div", class_ = "col-sm-2 text-right result-amount").text.replace("Bevilget beløb", "").strip()
        
        modtager_liste = bevilling.find("div", class_ = "col-xs-6 col-sm-12").text.strip().split(sep="\n\n") # remove whitespaces and split based on "\n\n"
        modtager_liste = [s.strip() for s in modtager_liste] # strip whitespace for both Modtager and Institution

        try:
            modtager = modtager_liste[0]
            institution = modtager_liste[1]
        except IndexError:
            modtager = modtager_liste[0]
            institution = ""
        
        info_liste = bevilling.find("ul", class_ = "listing-horizontal").text.strip().split(sep="\n\n") # remove whitespaces and split based on "\n\n"
        info_liste = [s.strip() for s in info_liste] # strip whitespace for both Virkemiddel, Område and År

        virkemidler = info_liste[0]
        område = info_liste[1]
        periode = info_liste[2]

        try:
            beskrivelse = bevilling.find("div", class_ = "row result-body").text.strip()
        except AttributeError:
            beskrivelse = ""

        # append rows to an empty DataFrame
        df = df.append({'Titel'             : titel,
                        'Bevilliget beløb'  : beløb,
                        'Modtager'          : modtager,
                        'Institution'       : institution,
                        'Virkemidler'       : virkemidler,
                        'Område'            : område,
                        'År'                : periode,
                        'Beskrivelse'       : beskrivelse},
                        ignore_index = True)

Scraping page: 0
Scraping page: 1
Scraping page: 2
Scraping page: 3
Scraping page: 4
Scraping page: 5
Scraping page: 6
Scraping page: 7
Scraping page: 8
Scraping page: 9
Scraping page: 10
Scraping page: 11
Scraping page: 12
Scraping page: 13
Scraping page: 14
Scraping page: 15
Scraping page: 16
Scraping page: 17
Scraping page: 18
Scraping page: 19
Scraping page: 20
Scraping page: 21
Scraping page: 22
Scraping page: 23
Scraping page: 24
Scraping page: 25
Scraping page: 26
Scraping page: 27
Scraping page: 28
Scraping page: 29
Scraping page: 30
Scraping page: 31
Scraping page: 32
Scraping page: 33
Scraping page: 34
Scraping page: 35
Scraping page: 36
Scraping page: 37
Scraping page: 38
Scraping page: 39
Scraping page: 40
Scraping page: 41
Scraping page: 42
Scraping page: 43
Scraping page: 44
Scraping page: 45
Scraping page: 46
Scraping page: 47
Scraping page: 48
Scraping page: 49
Scraping page: 50
Scraping page: 51
Scraping page: 52
Scraping page: 53
Scraping page: 54
Scraping page: 55
Sc

In [9]:
df.tail()

,Titel,Bevilliget beløb,Modtager,Institution,Virkemidler,Område,År,Beskrivelse
4426,Enabling Ultra Deep Hydrodesulphurization by N...,10.781.874 kr,Ib Chorkendorff,Danmarks Tekniske Universitet,Øvrige forskningsprojekter,Teknologi og Produktion,2013,Alle olieprodukter renses i dag for svovl for ...
4427,Acute stroke research,717.359 kr,Hanne Krarup Christensen,"Bispebjerg Hospital, Neurologisk Afdeling",Delestillinger,Sundhed og Sygdom,2013,Aktuelle ansøgning angår frikøb af overlæge Ha...
4428,Atherosclerotic cardiovascular disease in HIV-...,764.683 kr,Anne-Mette Lebech,"Hvidovre Hospital, Infektionsmedicinsk Afdeling",Delestillinger,Sundhed og Sygdom,2013,Behandling af HIV positive patienter med anti-...
4429,Epigenetic modulation of mechanisms involved i...,829.294 kr,Ole Schmeltz Søgaard,"Aarhus Universitetshospital, Infektionsmedicin...",Delestillinger,Sundhed og Sygdom,2013,HIV infektion behandles i dag med en kombinati...
4430,Novel mechanisms of insulin resistance and mit...,665.923 kr,Kurt Højlund,"Odense Universitetshospital, Endokrinologisk A...",Delestillinger,Sundhed og Sygdom,2013,Insulinresistens (IR) i muskelvæv spiller en v...


In [10]:
# ------ Data Wrangling

# trim extra space between substrings
df["Institution"] = df["Institution"].str.replace(' +', ' ')

# universities - KU
df['Institution'] = df['Institution'].replace(['Købehavns Universitet', 'Københans Universitet', 'København Universitet'], 'Københavns Universitet') # misspelling
df.loc[df['Institution'].str.contains(pat='KU'), 'Institution'] = 'Københavns Universitet' # abbreviations
df.loc[df['Institution'].str.contains(pat='University of Copenhagen'), 'Institution'] = 'Københavns Universitet' # english to danish

# universities - DTU
df['Institution'] = df['Institution'].replace(['Danmarks Teknisk Universitet'], 'Danmarks Tekniske Universitet') # misspelling
df.loc[df['Institution'].str.contains(pat='DTU'), 'Institution'] = 'Danmarks Tekniske Universitet' # abbreviations
df.loc[df['Institution'].str.contains(pat='Technical University of Denmark'), 'Institution'] = 'Danmarks Tekniske Universitet' # english to danish

# universities – CBS
df.loc[df['Institution'].str.contains(pat='Handelshøjskole'), 'Institution'] = 'Copenhagen Business School' # danish to english
df.loc[df['Institution'].str.contains(pat='Copenhagen Buisness School'), 'Institution'] = 'Copenhagen Business School'

# universities – RUC
df.loc[df['Institution'].str.contains(pat='Roskilde University'), 'Institution'] = 'Roskilde Universitet'

# universities – AU
df.loc[df['Institution'].str.contains(pat=r"Aarhus University[^a-z]*$"), 'Institution'] = 'Aarhus Universitet' # does not accept "... University Hospital", but misses "," and "/"
df.loc[df['Institution'].str.contains(pat=r"Aarhus Universitet[^a-z]*$"), 'Institution'] = 'Aarhus Universitet' # does not accept "... Universitetshopsital", but misses "," and "/"
df.loc[df['Institution'].str.contains(pat="Aarhus Universitet,"), 'Institution'] = 'Aarhus Universitet'
df.loc[df['Institution'].str.contains(pat="Aarhus Universitet/"), 'Institution'] = 'Aarhus Universitet'
df.loc[df['Institution'].str.contains(pat="Aarhus University,"), 'Institution'] = 'Aarhus Universitet'
df.loc[df['Institution'].str.contains(pat="Aarhus University/"), 'Institution'] = 'Aarhus Universitet'

# universities – AAU
df.loc[df['Institution'].str.contains(pat="Aalborg Universitet/"), 'Institution'] = 'Aalborg Universitet'
df.loc[df['Institution'].str.contains(pat="Aalborg Universitet,"), 'Institution'] = 'Aalborg Universitet'
df.loc[df['Institution'].str.contains(pat="Aalborg University"), 'Institution'] = 'Aalborg Universitet'

# universities – SDU
df.loc[df['Institution'].str.contains(pat=r"^Syddansk Universitet"), 'Institution'] = 'Syddansk Universitet' # starts with pat
df.loc[df['Institution'].str.contains(pat=r"Syddansk Universitet$"), 'Institution'] = 'Syddansk Universitet' # ends with pat

# university hospital - AAU
df.loc[df['Institution'].str.contains(pat="Aalborg Universitetshospital"), 'Institution'] = 'Aalborg Universitetshospital' # match "Aarhus Universitet"

# university hospital - AU
df.loc[df['Institution'].str.contains(pat="Aarhus Universitetshospital,"), 'Institution'] = 'Aarhus Universitetshospital'
df.loc[df['Institution'].str.contains(pat="Aarhus University Hospital"), 'Institution'] = 'Aarhus Universitetshospital'
df.loc[df['Institution'].str.contains(pat="Aarhus Universitetshospital"), 'Institution'] = 'Aarhus Universitetshospital'
df.loc[df['Institution'].str.contains(pat="Aarhus Universitshospital,"), 'Institution'] = 'Aarhus Universitetshospital'

# university hospital - OUH
df.loc[df['Institution'].str.contains(pat="Odense Universitets Hospital"), 'Institution'] = 'Odense Universitetshospital'
df.loc[df['Institution'].str.contains(pat="Odense University Hospital"), 'Institution'] = 'Odense Universitetshospital'

# GEUS, VIVE, SFI
df.loc[df['Institution'].str.contains(pat="GEUS"), 'Institution'] = 'De Nationale Geologiske Undersøgelser for Danmark og Grønland'
df.loc[df['Institution'].str.contains(pat="VIVE"), 'Institution'] = 'Det Nationale Forsknings- og Analysecenter for Velfærd'
df.loc[df['Institution'].str.contains(pat="SFI"), 'Institution'] = 'Det Nationale Forsknings- og Analysecenter for Velfærd'


# convert column År to numeric
df["År"] = pd.to_numeric(df["År"])

# convert column Bevilliget beløb to numeric
df["Bevilliget beløb"] = df["Bevilliget beløb"].str.replace("kr", "").str.strip()
df["Bevilliget beløb"] = df["Bevilliget beløb"].str.replace(".", "")
df["Bevilliget beløb"] = pd.to_numeric(df["Bevilliget beløb"])

# convert first character of each word to uppercase in column Institution
df["Institution"] = df["Institution"].str.title()

In [11]:
# sense check
df.dtypes

Titel               object
Bevilliget beløb     int64
Modtager            object
Institution         object
Virkemidler         object
Område              object
År                   int64
Beskrivelse         object
dtype: object

In [12]:
# create dictionary for Region
regioner_dict = {
    "Region Nordjylland": [ "Aalborg", "Nordjylland", "Nordjyllands", "Ålborg"],

    "Region Midtjylland": [ "Aarhus", "Midtjylland", "Midtjyllands", "Via University College", "Danmarks Industrimuseum",
                            "Horsens", "Århus", "Central Region Denmark", "KORA", "Moesgård Museum",
                            "Center for Rusmiddelforskning"],

    "Region Syddanmark" : [ "Syddansk", "Southern Denmark", "Odense", "Kolding", "Syddanmark", "Syddanmarks",
                            "Dansk Sprognævn"],

    "Region Sjælland"   : [ "Sjælland", "Roskilde", "Slagelse", "Professionshøjskolen Absalon", "Dansk Historisk Fællesråd",
                            "Sjællands", "Lolland", "Falster", "Næstved", "Holbæk", "Nykøbing"],

    "Region Hovedstaden": [ "København", "Copenhagen", "Københavns", "Nordsjælland", "Nordsjællands",
                            "Danmarks Tekniske Universitet", "Hovedstaden", "Hovedstadens","Rigshospitalet", "Bispebjerg",
                            "Herlev", "Gentofte", "Hvidovre", "Glostrup", "Frederiksberg", "Hillerød",
                            "Forsvarsakademiet", "Statens Serum Institut", "Ssi",
                            "Det Kongelige Danske", "Kadk", "Det Nationale Forsknings- Og Analysecenter For Velfærd", "Statens Museum For Kunst", "Smk",
                            "University College Capital", "Ucc", "Center For Kræftforskning",
                            "Diis", "Dignity", "Selskabet for Skole- Og Uddannelseshistorie",
                            "Statens Naturhistoriske Museum", "Professionshøjskolen Metropol", "Kræftens Bekæmpelse",
                            "Dansk Institut Mod Tortur", "Rigsarkivet", "Det Kongelige Akademi", "Psykiatrisk Center Sct. Hans",
                            "De Nationale Geologiske Undersøgelser For Danmark Og Grønland", "Nationalmuseet",
                            "Dansk Institut For Internationale Studier", "Den Selvejende Institution Dansk Børneastma Center",
                            "Institut For Menneskerettigheder", "Det Kongelige Bibliotek", "Danish archives, museums and libraries",
                            "Det Nationale Forskningscenter For Arbejdsmiljø", "Forskningscenter For Forebyggelse Og Sundhed",
                            "Rockwool Fonden", "Selskabet Til Forskning I Arbejderbevægelsens Historie",
                            "Universitets-Jubilæets Danske Samfund", "Dhi", "Statens Arkiver",
                            "Det Nationale Institut For Kommuners Og Regioners Analyse Og Forskning"]
}

In [13]:
# create empty column for Region
df["Region"] = np.nan

# compares dict values to Institution string and append with key
for i in range(len(df["Institution"])): # loops over instution
    
    for substring_list in regioner_dict.values(): # loops over list of values in dictionary
        
        if any(substring in df["Institution"][i] for substring in substring_list) == True: # if institution matches any of the substrings then find the key
            region = list(regioner_dict.keys())[list(regioner_dict.values()).index(substring_list)] #
            df["Region"][i] = region

# sense check
df.head()

C:\Users\gusta\AppData\Local\Temp\ipykernel_25532\3759746254.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Region"][i] = region


,Titel,Bevilliget beløb,Modtager,Institution,Virkemidler,Område,År,Beskrivelse,Region
0,"A relativistic, virtual lab for light-driven a...",6189120,Erik Donovan Hedegård,Syddansk Universitet,Sapere Aude Forskningsleder,Tværrådslig,2022,Kræft er blandt de hyppigste dødsårsager i EU ...,Region Syddanmark
1,ACHiLLES: ACtive Human Labeling and LEarning S...,6184075,Dimitrios Papadopoulos,Danmarks Tekniske Universitet,Sapere Aude Forskningsleder,Tværrådslig,2022,Vi lever i en spændende æra for computer visio...,Region Hovedstaden
2,ATEMPO: Anthropogenic impacts on Temporal Biod...,6189551,Naia Morueta Holme,Københavns Universitet,Sapere Aude Forskningsleder,Tværrådslig,2022,"Vi lever i Menneskets Tidsalder, Antropocæn. M...",Region Hovedstaden
3,Adaptation of T cell immunity in viral infecti...,6168120,Sunil Kumar Saini,Danmarks Tekniske Universitet,Sapere Aude Forskningsleder,Tværrådslig,2022,Igennem dette projekt vil vi undersøge hvordan...,Region Hovedstaden
4,Adapting to cope with unpredictable climates o...,6189093,Mads Fristrup Schou,Aarhus Universitet,Sapere Aude Forskningsleder,Tværrådslig,2022,For at fastslå hvordan dyr vil reagere på klim...,Region Midtjylland


In [15]:
nan_df = df[df['Region'].isnull()]
nan_df

,Titel,Bevilliget beløb,Modtager,Institution,Virkemidler,Område,År,Beskrivelse,Region
242,DKOPT – DEF - Voter behavior in the Danish def...,491040,Derek John Beach,Uoplyst,Forskningsprojekt 1,Samfund og Erhverv,2022,"Hvordan beslutter vælgere, hvad de vil stemme ...",NaN
436,Journal of Business Anthropology,120000,Kasper Tang Vangkilde,,Tidsskrifter,Kultur og Kommunikation,2021,Tidsskriftet Journal of Business Anthropology ...,NaN
437,K&K - Kultur og klasse,120000,Jonas Ross Kjærgård,,Tidsskrifter,Kultur og Kommunikation,2021,"K&K – Kultur og Klasse er et humanistisk, fagf...",NaN
438,Peripeti - tidsskrift for dramaturgiske studier,90000,Thomas Rosendal Nielsen,,Tidsskrifter,Kultur og Kommunikation,2021,Peripeti er et videnskabeligt tidsskrift for d...,NaN
439,Research and Change,120000,Janne Hedegaard Hansen,,Tidsskrifter,Kultur og Kommunikation,2021,Tidsskriftet Forskning og Forandring er et dou...,NaN
...,...,...,...,...,...,...,...,...,...
4105,Pre-Clinical Testing of Immune Cells Engineere...,2468080,Rasmus Otkjær Bak,Stanford University,Tidligere virkemiddel » Individuelle postdoc,Sundhed og Sygdom,2013,Trods 30 års intens forskning i HIV findes der...,NaN
4117,WNT signaling in dopamine differenciation of s...,1255802,Fabia Febbraro,Karolinska Institutet,Tidligere virkemiddel » Individuelle postdoc,Sundhed og Sygdom,2013,Parkinsons sygdom (PS) er én af de mest udbred...,NaN
4256,JHEP - Ageing Study of Treated Composite Archa...,741177,Martin Nordvig Mortensen,"Danish Archives, Museums And Libraries",Forskningsprojekt 1,Kultur og Kommunikation,2013,JHEP - Ageing Study of Treated Composite Archa...,NaN
4283,Ansøgning om tidsskriftsstøtte til Landbohisto...,90000,Mogens Rostgaard Nissen,Dansk Centralbibliotek For Sydslesvig,Tidsskrifter,Kultur og Kommunikation,2013,,NaN


In [16]:
# save to csv
df.to_csv('dff.csv', sep=',', header=True, index=False)